<a href="https://colab.research.google.com/github/divin-di/Final-year-project/blob/main/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import pandas as pd
import re
import nltk

#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import StandardScaler

# Load the dataset into a pandas DataFrame
df = pd.read_csv("/content/NYC_Jobs.csv")

# Define a function to clean and preprocess the job description text data
def preprocess_text(text):
    # Remove URLs and HTML tags
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    
    # Convert the text to lowercase
    text = text.lower()
    
    # Tokenize the text into individual words
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if not token in stop_words]
    
    # Stem the remaining words
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Remove any remaining punctuation and non-alphabetic characters
    tokens = [re.sub(r'[^a-zA-Z]', '', token) for token in tokens]
    
    # Remove any tokens that are only one character long
    tokens = [token for token in tokens if len(token) > 1]
    
    # Join the remaining tokens back into a single string
    text = ' '.join(tokens)
    
    return text

# Apply the preprocess_text function to the 'Job Description' column of the DataFrame
df['Job Description'] = df['Job Description'].apply(preprocess_text)
#print(df)


# Apply t-SNE to the normalized data
#tsne = TSNE(n_components=3,verbose=1, perplexity = 80, n_iter= 1000,learning_rate=200, random_state=42)


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Load the dataset into a pandas DataFrame
#df = pd.read_csv(r"C:\Users\Divin\Desktop\final year project\NYC_Jobs.csv")
features = ['Work Location', 'Job Description', 'Salary Range From']
data = pd.get_dummies(df[features])
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
# Clean and preprocess the text data (not shown)
#print(data)
#print(data_scaled)

# Convert the preprocessed text data into a matrix of numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Job Description'])

# Apply t-SNE to the feature matrix to reduce the dimensionality of the data to 2 dimensions
tsne = TSNE(n_components=3,verbose=1, perplexity = 80,learning_rate=200,n_iter=2000, random_state=42)
X_tsne = tsne.fit_transform(X.toarray())

tsne_scale_results = tsne.fit_transform(data_scaled)

tsne_df_scale = pd.DataFrame(tsne_scale_results, columns=['tsne1', 'X_tsne', 'tsne3'])




#plt.figure(figsize = (10,10))
#plt.scatter(tsne_df_scale.iloc[:,0],tsne_df_scale.iloc[:,1],alpha=0.25, facecolor='lightslategray')
#plt.xlabel('tsne1')
#plt.ylabel('tsne2')
#plt.show()
#print(tsne1)

# Visualize the reduced-dimensional data using a scatter plot

#import numpy as np
#import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

#df = pd.read_csv(r"C:\Users\Divin\Desktop\final year project\NYC_Jobs.csv")
#kmeans=KMeans(n_clusters=10)
#kmeans.fit(X_tsne)
#plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=kmeans.labels_, cmap='rainbow')
##plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=kmeans.labels_, cmap='rainbow')
#plt.colorbar()
#plt.show()
#print(X_tsne)



sse = []
k_list = range(1, 15)
for k in k_list:
    km = KMeans(n_clusters=k)
    km.fit(tsne_df_scale)
    sse.append([k, km.inertia_])
    
tsne_results_scale = pd.DataFrame({'Cluster': range(1,15), 'SSE': sse})
#plt.figure(figsize=(12,6))
#plt.plot(pd.DataFrame(sse)[0], pd.DataFrame(sse)[1], marker='o')
#plt.title('Optimal Number of Clusters using Elbow Method (tSNE_Scaled Data)')
#plt.xlabel('Number of clusters')
#plt.ylabel('Inertia')

kmeans_tsne_scale = KMeans(n_clusters=10, n_init=100, max_iter=100, init='k-means++', random_state=42).fit(tsne_df_scale)
#print('KMeans tSNE Scaled Silhouette Score: {}'.format(silhouette_score(tsne_df_scale, kmeans_tsne_scale.labels_, metric='euclidean')))
labels_tsne_scale = kmeans_tsne_scale.labels_
clusters_tsne_scale = pd.concat([tsne_df_scale, pd.DataFrame({'tsne_clusters':labels_tsne_scale})], axis=1)

import seaborn as sns
#plt.figure(figsize = (15,15))
#sns.scatterplot(clusters_tsne_scale.iloc[:,0],clusters_tsne_scale.iloc[:,1],hue=labels_tsne_scale, palette='Set1', s=100, alpha=0.6).set_title('Cluster Vis tSNE Scaled Data', fontsize=15)
#plt.legend()
#plt.show()


import plotly.graph_objs as go
Scene = dict(xaxis = dict(title  = 'Work location'),yaxis = dict(title  = 'Job Description'),zaxis = dict(title  = 'Salary Range From'))
labels = labels_tsne_scale
trace = go.Scatter3d(x=clusters_tsne_scale.iloc[:,0], y=clusters_tsne_scale.iloc[:,1], z=clusters_tsne_scale.iloc[:,2], mode='markers',marker=dict(color = labels, colorscale='Viridis', size = 2, line = dict(color = 'yellow',width = 5)))
layout = go.Layout(margin=dict(l=0,r=0),scene = Scene, height = 1000,width = 1000)
data = [trace]
fig = go.Figure(data = data, layout = layout)
fig.show()


[t-SNE] Computing 241 nearest neighbors...
[t-SNE] Indexed 6085 samples in 0.045s...
[t-SNE] Computed neighbors for 6085 samples in 17.337s...
[t-SNE] Computed conditional probabilities for sample 1000 / 6085
[t-SNE] Computed conditional probabilities for sample 2000 / 6085
[t-SNE] Computed conditional probabilities for sample 3000 / 6085
[t-SNE] Computed conditional probabilities for sample 4000 / 6085
[t-SNE] Computed conditional probabilities for sample 5000 / 6085
[t-SNE] Computed conditional probabilities for sample 6000 / 6085
[t-SNE] Computed conditional probabilities for sample 6085 / 6085
[t-SNE] Mean sigma: 0.551049
[t-SNE] KL divergence after 250 iterations with early exaggeration: 79.010292
[t-SNE] KL divergence after 2000 iterations: 0.449470
[t-SNE] Computing 241 nearest neighbors...
[t-SNE] Indexed 6085 samples in 0.096s...
[t-SNE] Computed neighbors for 6085 samples in 5.104s...
[t-SNE] Computed conditional probabilities for sample 1000 / 6085
[t-SNE] Computed condition

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: F

In [12]:
pip install scikit-learn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 85.3 MB/s eta 0:00:00


In [6]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import nltk

nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True